In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

In [4]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import revsic_glowtts as glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [6]:
i = tf.convert_to_tensor([data['text_ids'][0],data['text_ids'][0]])
i_lengths = tf.convert_to_tensor([data['len_text_ids'][0,0], data['len_text_ids'][0,0]])
mel_outputs = tf.convert_to_tensor([data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]])
mel_lengths = tf.convert_to_tensor([408,408])

In [7]:
config = glowtts.Config(mel = 80, vocabs = 66)

In [8]:
model = glowtts.Model(config)

In [9]:
loss, losses, attn = model.compute_loss(text = i, textlen = i_lengths, mel = mel_outputs, mellen = mel_lengths)
loss, losses, attn


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(<tf.Tensor: id=29117, shape=(), dtype=float32, numpy=0.82437336>,
 {'nll': <tf.Tensor: id=28923, shape=(), dtype=float32, numpy=0.31569207>,
  'durloss': <tf.Tensor: id=29116, shape=(), dtype=float32, numpy=0.5086813>},
 <tf.Tensor: id=28903, shape=(2, 204, 72), dtype=float32, numpy=
 array([[[1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.]],
 
        [[1., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>)

In [10]:
mel, mellen, attn = model(inputs = i, lengths = i_lengths)
mel, mellen, attn

(<tf.Tensor: id=35932, shape=(2, 426, 80), dtype=float32, numpy=
 array([[[ 0.0299217 ,  0.19960961,  0.20560375, ..., -0.16675694,
          -0.15730046, -0.25934842],
         [ 0.02185097,  0.20416486,  0.22282882, ..., -0.18458278,
          -0.17118862, -0.270863  ],
         [ 0.0299217 ,  0.19960961,  0.20560375, ..., -0.16675694,
          -0.15730046, -0.25934842],
         ...,
         [ 0.        ,  0.        ,  0.        , ..., -0.        ,
          -0.        , -0.        ],
         [ 0.        ,  0.        ,  0.        , ..., -0.        ,
          -0.        , -0.        ],
         [ 0.        ,  0.        ,  0.        , ..., -0.        ,
          -0.        , -0.        ]],
 
        [[ 0.0299217 ,  0.19960961,  0.20560375, ..., -0.16675694,
          -0.15730046, -0.25934842],
         [ 0.02185097,  0.20416485,  0.22282882, ..., -0.18458278,
          -0.17118862, -0.270863  ],
         [ 0.0299217 ,  0.19960962,  0.20560375, ..., -0.16675694,
          -0.157300